In [50]:
# import libraries
import pandas as pd
import numpy as np 
import warnings
import gmplot
import sys
import geopy
#from geopy import distance
from copy import deepcopy
sys.path.append("../HistoricalData/")
from getData import get_data

warnings.filterwarnings('ignore')

In [4]:
# this call to get_data function that will take bounding box and timeframe and return cleaned data

UP_LEFT = (38.008050, -122.536985)    # (lat, lon)
UP_RIGHT = (38.008050, -122.186437)   # (lat, lon)
DOWN_RIGHT = (37.701933, -122.186437) # (lat, lon)
DOWN_LEFT = (37.701933, -122.536985)  # (lat, lon)
START_DATE = '2019/09/01' # begin date to start taking data
END_DATE = '2019/09/10'   # end date to start taking data
START_HOUR = '0'         # hour EACH DAY to start, this allows us to control for time of day effects
END_HOUR = '1'           # hour EACH DAY to end, this allows us to control for time of day effects
color = "red"

data_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR)

In [8]:
data_df.columns

Index(['0_3um', '0_5um', '1_0um', '2_5um', '5_0um', '10_0um', 'pm1_0',
       'pm10_0', 'created', 'pm1_0_atm', 'pm2_5_atm', 'pm10_0_atm', 'uptime',
       'rssi', 'temperature', 'humidity', 'pm2_5_cf_1', 'device_loc_typ',
       'is_owner', 'sensor_id', 'sensor_name', 'parent_id', 'lat', 'lon',
       'thingspeak_primary_id', 'thingspeak_primary_id_read_key',
       'thingspeak_secondary_id', 'thingspeak_secondary_id_read_key', 'a_h',
       'high_reading_flag', 'hidden', 'city', 'county', 'zipcode',
       'created_at', 'year', 'month', 'day', 'hour', 'minute', 'wban_number',
       'call_sign', 'call_sign2', 'interval', 'call_sign3', 'zulu_time',
       'report_modifier', 'wind_data', 'wind_direction', 'wind_speed', 'gusts',
       'gust_speed', 'variable_winds', 'variable_wind_info', 'sys_maint_reqd',
       'epa_pm25_unit', 'epa_pm25_value', 'raw_concentration', 'aqi',
       'category', 'site_name', 'agency_name', 'full_aqs_code',
       'intl_aqs_code'],
      dtype='object')

In [26]:
type(data_df.iloc[0].created)

int

In [30]:
type(data_df.iloc[0].sensor_id)

str

In [42]:
data_df.iloc[0].wind_direction

'280'

In [14]:
data_df.groupby(data_df.sensor_id).count()

,0_3um,0_5um,1_0um,2_5um,5_0um,10_0um,pm1_0,pm10_0,created,pm1_0_atm,...,sys_maint_reqd,epa_pm25_unit,epa_pm25_value,raw_concentration,aqi,category,site_name,agency_name,full_aqs_code,intl_aqs_code
sensor_id,,,,,,,,,,,,,,,,,,,,,
1004,600,600,600,600,600,600,600,600,600,600,...,492,600,600,600,600,600,600,600,600,600
1005,600,600,600,600,600,600,600,600,600,600,...,492,600,600,600,600,600,600,600,600,600
10066,212,212,212,212,212,212,212,212,430,212,...,324,430,430,430,430,430,430,430,430,430
10067,201,201,201,201,201,201,201,201,424,207,...,318,424,424,424,424,424,424,424,424,424
1014,600,600,600,600,600,600,600,600,600,600,...,492,600,600,600,600,600,600,600,600,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,600,600,600,600,600,600,600,600,600,600,...,491,540,540,540,540,540,540,540,540,540
7198,494,494,494,494,494,494,494,494,582,494,...,476,582,582,582,582,582,582,582,582,582
7199,494,494,494,494,494,494,494,494,582,494,...,476,582,582,582,582,582,582,582,582,582


In [13]:
data_df.groupby(data_df.created).count()

,0_3um,0_5um,1_0um,2_5um,5_0um,10_0um,pm1_0,pm10_0,pm1_0_atm,pm2_5_atm,...,sys_maint_reqd,epa_pm25_unit,epa_pm25_value,raw_concentration,aqi,category,site_name,agency_name,full_aqs_code,intl_aqs_code
created,,,,,,,,,,,,,,,,,,,,,
201909010000,487,487,487,487,487,487,487,487,487,487,...,487,487,487,487,487,487,487,487,487,487
201909010010,487,487,487,487,487,487,487,487,487,487,...,487,487,487,487,487,487,487,487,487,487
201909010020,487,487,487,487,487,487,487,487,487,487,...,487,487,487,487,487,487,487,487,487,487
201909010030,487,487,487,487,487,487,487,487,487,487,...,487,487,487,487,487,487,487,487,487,487
201909010040,487,487,487,487,487,487,487,487,487,487,...,487,487,487,487,487,487,487,487,487,487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201909100910,496,496,496,496,496,496,496,496,498,498,...,504,504,504,504,504,504,504,504,504,504
201909100920,496,496,496,496,496,496,496,496,496,496,...,504,504,504,504,504,504,504,504,504,504
201909100930,497,497,497,497,497,497,497,497,495,495,...,504,504,504,504,504,504,504,504,504,504


In [101]:
class Chain:
    """ 
    Chain is a class that defines a chain of sensors in time and space. 
    A chain has a list of timestamps that are paired with a list of particle locations, intersecting sensors, and readings
    """
    def __init__(self, sensor_id, start_time):
        self.start_time = start_time
        self.time = [self.start_time]
        self.sensor_path = [sensor_id]
        self.last_sensor = sensor_id
        self.particle_path = [(data_df[data_df.sensor_id == sensor_id].lat[0], data_df[data_df.sensor_id == sensor_id].lon[0])]
        self.readings = [data_df[(data_df.sensor_id == sensor_id) & (data_df.created == start_time)]['2_5um']]
    
    def plot_next(self):
        # update time
        MINUTES = 10 # Joined data seems to be in increments of 10 minutes
        origin_time = self.time[-1]
        destination_time = self.time[-1] + MINUTES
        self.time.append(destination_time)
        
        # need to take wind speed in knots
        origin = geopy.Point(self.particle_path[-1][0], self.particle_path[-1][1])
        km_1_knot_per_hour = 1.852
        km_hour = km_1_knot_per_hour * data_df[(data_df.sensor_id == self.last_sensor) & (data_df.created == origin_time)].wind_speed[0] # so you are grabbing from last sensor seen... might not be nearest wind data
 
        d = (MINUTES / 60) * km_hour 

        print(km_hour, d)
    
        # convert wind_direction to a bearing
        bearing = data_df[(data_df.sensor_id == self.last_sensor) & (data_df.created == origin_time)].wind_direction 
        try:
            b = int(bearing) + 180
        except: # 'VRB' = variable winds
            return false 
        if b >= 360:
            b -= 360
        
        # given: lat1, lon1, b = bearing in degrees, d = distance in kilometers
        if d == 0:
            destination = self.particle_path[-1]
        else:
            destination = geopy.distance.VincentyDistance(kilometers=d).destination(origin, b)
            destination = (destination.latitude, destination.longitude)
        
            # end chain if outside bounding box
            if (destination[0] > UP_LEFT[0]) or (destination[0] < DOWN_LEFT[0]) or (destination[1] < UP_LEFT[1]) or (destination[1] > DOWN_RIGHT[1]):
                return false
        
            self.particle_path.append(destination)
        
            # check is there's a sensor nearby
            CLOSE_ENOUGH_DISTANCE = 0.05 # have to be within 50 meters
        
            closest_sensor = None
            for sensor in data_df.sensor_id.unique(): # iterate through all sensors in bounding box
                # debug print
                print("Sensor", sensor)
                print((float(data_df[data_df.sensor_id == sensor].lat[0]), float(data_df[data_df.sensor_id == sensor].lon[0])))
                print(destination)
                test_sensor = (float(data_df[data_df.sensor_id == sensor].lat[0]), float(data_df[data_df.sensor_id == sensor].lon[0]))
                testy = geopy.point.Point(test_sensor)
                desty = geopy.point.Point(destination)
                proximity = geopy.distance.distance(testy, desty).km
                if proximity < CLOSE_ENOUGH_DISTANCE:
                    closest_sensor = sensor.sensor_id
                
            # update other lists
            self.sensor_path.append(closest_sensor)
            if closest_sensor:
                self.last_sensor = closest_sensor
                self.readings.append(data_df[(data_df.sensor_id == closest_sensor) & (data_df.created == destination_time)]['2_5um'])

        return true
    
    def plot_pollution(self):
        return 

In [102]:
chains = [] # a list of chain 

set_size_chain = 10
start_time = 201909010000

# build the chains
for sensor in data_df.sensor_id.unique(): # iterate through all sensors in bounding box
    print(sensor) # debugging
    chain = Chain(sensor, start_time) # init one chain per sensor
    time_step = 0
    while (chain.plot_next() & (time_step < set_size_chain)): # method returns false if out of bounds/data
        time_step += 1
    chains.append(chain) # add the chain object to the list of chains

16939
12.964 2.1606666666666667
Sensor 16939


TypeError: cannot convert the series to <class 'float'>

In [61]:
# plot it
latitude_list = []
longitude_list = []
for row in range(len(model_df)):
    try:
        latitude_list.append(model_df.loc[row].lat)
        longitude_list.append(model_df.loc[row].lon)
    except:
        pass
gmap3 = gmplot.GoogleMapPlotter((max(latitude_list)+min(latitude_list))/2,
                                (max(longitude_list)+min(longitude_list))/2,
                                zoom=11)
gmap3.scatter(latitude_list, longitude_list, '# FF0000', 
                              size = 40, marker = False ) 

for chain in chains:
    lats, lons, color = chain.plot_pollution()
    gmap3.polygon(lats, lons, edge_color=color, edge_width=1, face_color=color, face_alpha=0.1)

# draw the map and send to file
gmap3.draw("../data/model_map.html") 
print("Done")

NameError: name 'model_df' is not defined